In [218]:
import numpy as np
import pandas as pd
import os
import spacy

In [219]:
datapath = "/Users/wasa/pyRandom/src/data/"
outputpath = "/Users/wasa/pyRandom/src/output/"
print(datapath)
print(outputpath)
                

/Users/wasa/pyRandom/src/data/
/Users/wasa/pyRandom/src/output/


In [220]:
train_df = pd.read_csv(datapath + "sfc_pn_train.csv", header=None)
train_df.columns = ["gender", "year", "kind", "opinion"]
test_df = pd.read_csv(datapath + "sfc_pn_test.csv", header=None)
test_df.columns = ["gender", "year", "kind", "opinion"]
train_df.head(5)

,gender,year,kind,opinion
0,male,3rd,positive,慶應生でもあり、SFC生でもあること。就活などで慶應生に会うと親近感が湧いてくる。だが、SF...
1,female,1st,negative,外国語のインテンシブが普通の授業よりも１０分長く、全く集中が続かない。しかも休み時間は変わら...
2,male,3rd,positive,研究会は研究会等名前であるが実際はサークルのようなものであり、気軽に出入りすることができる、...
3,male,1st,negative,SFCは自然が豊かであるがために、虫がキャンパス内を横行する。朝はそんなに見かけることはない...
4,male,1st,positive,キャンパスの雰囲気がよいところ。郊外に位置し駅からも遠いため閑静であり、敷地は開放感があり緑...


In [221]:
pos_df = train_df[train_df['kind'] == 'positive'].reset_index()
neg_df = train_df[train_df['kind'] == 'negative'].reset_index()

In [222]:
pos_df.head(5)

,index,gender,year,kind,opinion
0,0,male,3rd,positive,慶應生でもあり、SFC生でもあること。就活などで慶應生に会うと親近感が湧いてくる。だが、SF...
1,2,male,3rd,positive,研究会は研究会等名前であるが実際はサークルのようなものであり、気軽に出入りすることができる、...
2,4,male,1st,positive,キャンパスの雰囲気がよいところ。郊外に位置し駅からも遠いため閑静であり、敷地は開放感があり緑...
3,6,male,3rd,positive,研究室が充実している。いろいろな施設があって学生証あればパソコンのある部屋に自由に出入りでき...
4,8,male,1st,positive,慶應義塾大学湘南藤沢キャンパスに通う生徒の個性が豊かであり、様々な分野の活動が見受けられる。...


In [223]:
neg_df.head(5)

,index,gender,year,kind,opinion
0,1,female,1st,negative,外国語のインテンシブが普通の授業よりも１０分長く、全く集中が続かない。しかも休み時間は変わら...
1,3,male,1st,negative,SFCは自然が豊かであるがために、虫がキャンパス内を横行する。朝はそんなに見かけることはない...
2,5,male,1st,negative,交通アクセスが不便です。湘南台駅からバス、自転車を使わないと学校にはつけない距離です。一度駅...
3,7,male,1st,negative,キャンパスが都心から遠いため、通学にかなりの時間がかかるところ。さらに、駅からも遠い上に公共...
4,9,male,1st,negative,駅からキャンパスまでバスで移動しなければならないということである。バスの本数は多い方だと思う...


In [224]:
nlp = spacy.load('ja_ginza_nopn')

In [225]:
def parse(df, key):
    res = np.array([])
    for i in range(len(df.index)):
        for sent in nlp(df.loc[i][key].strip()).sents:
            for token in sent:
                res = np.append(res, token.lemma_)
                
#                 print(token.i)
#                 print(token.orth_)
#                 print(token.lemma_)
#                 print(token.pos_)
#                 print(token._.pos_detail)
    return res

In [226]:
pos_count = parse(pos_df, 'opinion')
pos_count_s = pd.Series(pos_count).value_counts()
word = pos_count_s.index
pos_count_df = pd.DataFrame({"word": word, "count" : pos_count_s.values})

In [227]:
neg_count = parse(neg_df, 'opinion')
neg_count_s = pd.Series(neg_count).value_counts()
word = neg_count_s.index
neg_count_df = pd.DataFrame({"word": word, "count" : neg_count_s.values})

In [228]:
all_count = parse(train_df, 'opinion')
all_count_s = pd.Series(all_count).value_counts()
word = all_count_s.index
all_count_df = pd.DataFrame({"word": word, "count" : all_count_s.values})

In [229]:
pos_count_df.info()
pos_count_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1375 entries, 0 to 1374
Data columns (total 2 columns):
word     1375 non-null object
count    1375 non-null int64
dtypes: int64(1), object(1)
memory usage: 21.6+ KB


,word,count
0,が,470
1,の,449
2,、,417
3,。,379
4,だ,359


In [230]:
neg_count_df.info()
neg_count_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1436 entries, 0 to 1435
Data columns (total 2 columns):
word     1436 non-null object
count    1436 non-null int64
dtypes: int64(1), object(1)
memory usage: 22.6+ KB


,word,count
0,が,512
1,の,442
2,、,417
3,。,399
4,に,333


In [231]:
all_count_df.info()
all_count_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2204 entries, 0 to 2203
Data columns (total 2 columns):
word     2204 non-null object
count    2204 non-null int64
dtypes: int64(1), object(1)
memory usage: 34.6+ KB


,word,count
0,が,982
1,の,891
2,、,834
3,。,778
4,だ,682


In [233]:
import math

In [274]:
merged_df = pd.merge(all_count_df, pos_count_df, on="word", how="left")
merged_df = pd.merge(merged_df, neg_count_df, on="word", how="left").fillna(0)
merged_df.columns = ["word", "all", "pos", "neg"]
merged_df['pos_prob'] = merged_df['pos'] / merged_df['all']
merged_df['neg_prob'] = merged_df['neg'] / merged_df['all']
merged_df['pos_prob'] = merged_df['pos_prob'].apply(lambda x: 0.01 if x < 0.01 else x )
merged_df['neg_prob'] = merged_df['neg_prob'].apply(lambda x: 0.01 if x < 0.01 else x )
merged_df['log_pos_prob'] = merged_df['pos_prob'].apply(lambda x : math.log(x))
merged_df['log_neg_prob'] = merged_df['neg_prob'].apply(lambda x : math.log(x))

In [275]:
merged_df

,word,all,pos,neg,pos_prob,neg_prob,log_pos_prob,log_neg_prob
0,が,982,470.0,512.0,0.478615,0.521385,-0.736859,-0.651267
1,の,891,449.0,442.0,0.503928,0.496072,-0.685322,-0.701035
2,、,834,417.0,417.0,0.500000,0.500000,-0.693147,-0.693147
3,。,778,379.0,399.0,0.487147,0.512853,-0.719190,-0.667765
4,だ,682,359.0,323.0,0.526393,0.473607,-0.641707,-0.747377
...,...,...,...,...,...,...,...,...
2199,可笑しい,1,0.0,1.0,0.010000,1.000000,-4.605170,0.000000
2200,経歴,1,1.0,0.0,1.000000,0.010000,0.000000,-4.605170
2201,片道,1,0.0,1.0,0.010000,1.000000,-4.605170,0.000000
2202,基準,1,0.0,1.0,0.010000,1.000000,-4.605170,0.000000


In [276]:
pos_prob = len(pos_df.index) / len(train_df.index)
neg_prob = len(neg_df.index) / len(train_df.index)

In [301]:
def classfication(prob_df, test_df):
    return_df = test_df.copy()
    return_df["predict_result"] = 0
    for i in range(len(test_df.index)):
        for sent in nlp(test_df.loc[i]['opinion'].strip()).sents:
            print("________")
            pos_probability = pos_prob
            neg_probability = neg_prob
            for token in sent:
                word = token.lemma_
                print(word)
                if word in prob_df['word'].values:
                    print('YES')
                    col = prob_df[prob_df['word'] == word]
                    print(col)
                    pos_probability += col['log_pos_prob'].values
                    neg_probability += col['log_neg_prob'].values
                    print(pos_probability)
                    print(neg_probability)
                    return_df.loc[i, 'predict_result'] = "positive" if pos_probability > neg_probability else "negative"
                    print(return_df.loc[i, 'predict_result'])
    return return_df
                
    

In [302]:
result = classfication(merged_df, test_df)

________
多種
YES
    word  all  pos  neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
417   多種    4  3.0  1.0      0.75      0.25     -0.287682     -1.386294
[0.21231793]
[-0.88629436]
positive
多様
YES
    word  all   pos  neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
172   多様   11  10.0  1.0  0.909091  0.090909      -0.09531     -2.397895
[0.11700775]
[-3.28418963]
positive
だ
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
4    だ  682  359.0  323.0  0.526393  0.473607     -0.641707     -0.747377
[-0.52469952]
[-4.03156697]
positive
人
YES
   word  all   pos   neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
31    人   76  46.0  30.0  0.605263  0.394737     -0.502092     -0.929536
[-1.02679147]
[-4.96110293]
positive
に
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
5    に  615  282.0  333.0  0.458537  0.541463     -0.779715      -0.61348
[-1.80650666]
[-5.5745827]
positive
出会う
YES
     word  all  pos  neg  pos_prob  n

   word  all    pos   neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
16  出来る  220  183.0  37.0  0.831818  0.168182     -0.184141      -1.78271
[-39.73633862]
[-49.83381732]
positive
。
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
3    。  778  379.0  399.0  0.487147  0.512853      -0.71919     -0.667765
[-40.45552894]
[-50.50158242]
positive
________
話
YES
    word  all  pos  neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
269    話    7  2.0  5.0  0.285714  0.714286     -1.252763     -0.336472
[-0.75276297]
[0.16352776]
negative
を
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
7    を  477  255.0  222.0  0.534591  0.465409     -0.626253     -0.764839
[-1.37901591]
[-0.60131135]
negative
聞く
YES
    word  all  pos  neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
189   聞く   10  5.0  5.0       0.5       0.5     -0.693147     -0.693147
[-2.07216309]
[-1.29445853]
negative
だけ
YES
   word  all   pos  neg  pos_prob  neg_

321   経験    6  4.0  2.0  0.666667  0.333333     -0.405465     -1.098612
[-29.64270828]
[-32.99136181]
positive
為る
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
9   為る  438  207.0  231.0  0.472603  0.527397       -0.7495     -0.639801
[-30.39220839]
[-33.63116301]
positive
こと
YES
   word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
11   こと  317  206.0  111.0  0.649842  0.350158     -0.431026     -1.049372
[-30.823234]
[-34.68053458]
positive
無い
YES
   word  all   pos   neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
22   無い  116  41.0  75.0  0.353448  0.646552     -1.040018     -0.436102
[-31.86325212]
[-35.11663666]
positive
入学
YES
    word  all  pos  neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
132   入学   15  9.0  6.0       0.6       0.4     -0.510826     -0.916291
[-32.37407775]
[-36.03292739]
positive
為る
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
9   為る  438  207.0  231.0  0.472603  0

     word  all  pos  neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
1046   とも    2  1.0  1.0       0.5       0.5     -0.693147     -0.693147
[-18.37870902]
[-23.5604237]
positive
に
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
5    に  615  282.0  333.0  0.458537  0.541463     -0.779715      -0.61348
[-19.15842421]
[-24.17390348]
positive
切磋琢磨
YES
      word  all  pos  neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
1805  切磋琢磨    1  1.0  0.0       1.0      0.01           0.0      -4.60517
[-19.15842421]
[-28.77907367]
positive
出来る
YES
   word  all    pos   neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
16  出来る  220  183.0  37.0  0.831818  0.168182     -0.184141      -1.78271
[-19.34256561]
[-30.5617833]
positive
。
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
3    。  778  379.0  399.0  0.487147  0.512853      -0.71919     -0.667765
[-20.06175593]
[-31.22954841]
positive
________
複数
YES
     word  all  pos  neg

________
SFC
YES
   word  all   pos   neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
21  SFC  118  59.0  59.0       0.5       0.5     -0.693147     -0.693147
[-0.19314718]
[-0.19314718]
negative
は
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
8    は  447  197.0  250.0  0.440716  0.559284     -0.819355     -0.581098
[-1.01250205]
[-0.77424486]
negative
良く
YES
    word  all  pos  neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
335   良く    6  1.0  5.0  0.166667  0.833333     -1.791759     -0.182322
[-2.80426152]
[-0.95656641]
negative
慶應大学
と
YES
   word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
10    と  320  126.0  194.0   0.39375   0.60625     -0.932039     -0.500463
[-3.7363006]
[-1.45702925]
negative
隔離
為る
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
9   為る  438  207.0  231.0  0.472603  0.527397       -0.7495     -0.639801
[-4.48580072]
[-2.09683045]
negative
れる
YES
   word  all   pos   n

...

   word  all   pos   neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
22   無い  116  41.0  75.0  0.353448  0.646552     -1.040018     -0.436102
[-20.23601421]
[-10.95351742]
negative
。
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
3    。  778  379.0  399.0  0.487147  0.512853      -0.71919     -0.667765
[-20.95520453]
[-11.62128253]
negative
________
辻堂
YES
    word  all  pos  neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
664   辻堂    3  0.0  3.0      0.01       1.0      -4.60517           0.0
[-4.10517019]
[0.5]
negative
を
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
7    を  477  255.0  222.0  0.534591  0.465409     -0.626253     -0.764839
[-4.73142313]
[-0.26483911]
negative
使う
YES
   word  all   pos   neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
72   使う   26  13.0  13.0       0.5       0.5     -0.693147     -0.693147
[-5.42457031]
[-0.95798629]
negative
て
YES
  word  all    pos    neg  pos_prob  neg_prob 

________
入学
YES
    word  all  pos  neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
132   入学   15  9.0  6.0       0.6       0.4     -0.510826     -0.916291
[-0.01082562]
[-0.41629073]
positive
時
YES
   word  all   pos   neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
47    時   44  18.0  26.0  0.409091  0.590909     -0.893818     -0.526093
[-0.9046435]
[-0.94238383]
positive
に
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
5    に  615  282.0  333.0  0.458537  0.541463     -0.779715      -0.61348
[-1.6843587]
[-1.55586361]
negative
専攻
YES
    word  all  pos  neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
591   専攻    3  3.0  0.0       1.0      0.01           0.0      -4.60517
[-1.6843587]
[-6.16103379]
positive
が
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
0    が  982  470.0  512.0  0.478615  0.521385     -0.736859     -0.651267
[-2.42121731]
[-6.81230047]
positive
決める
YES
    word  all  pos  neg  pos_prob  neg_

    word  all  pos  neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
464   全く    4  3.0  1.0      0.75      0.25     -0.287682     -1.386294
[-6.17274501]
[-12.99446572]
positive
興味
YES
   word  all   pos  neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
83   興味   22  19.0  3.0  0.863636  0.136364     -0.146603      -1.99243
[-6.31934848]
[-14.98689589]
positive
の
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
1    の  891  449.0  442.0  0.503928  0.496072     -0.685322     -0.701035
[-7.00467002]
[-15.68793043]
positive
無い
YES
   word  all   pos   neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
22   無い  116  41.0  75.0  0.353448  0.646552     -1.040018     -0.436102
[-8.04468815]
[-16.12403251]
positive
た
YES
   word  all   pos   neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
19    た  142  87.0  55.0  0.612676  0.387324     -0.489919     -0.948494
[-8.53460709]
[-17.07252638]
positive
分野
YES
   word  all   pos  neg  pos_prob  neg_prob  log_

negative
有る
YES
   word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
14   有る  307  133.0  174.0  0.433225  0.566775     -0.836499     -0.567792
[-9.00279147]
[-6.51212135]
negative
、
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
2    、  834  417.0  417.0       0.5       0.5     -0.693147     -0.693147
[-9.69593865]
[-7.20526853]
negative
近隣
YES
    word  all  pos  neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
852   近隣    2  0.0  2.0      0.01       1.0      -4.60517           0.0
[-14.30110884]
[-7.20526853]
negative
に
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
5    に  615  282.0  333.0  0.458537  0.541463     -0.779715      -0.61348
[-15.08082403]
[-7.8187483]
negative
農業
YES
    word  all  pos  neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
508   農業    3  1.0  2.0  0.333333  0.666667     -1.098612     -0.405465
[-16.17943632]
[-8.22421341]
negative
用
YES
     word  all  pos  neg  pos_p

763   中々    2  1.0  1.0       0.5       0.5     -0.693147     -0.693147
[-12.66885503]
[-6.67573628]
negative
無くなる
ない
YES
   word  all   pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
17   ない  190  45.0  145.0  0.236842  0.763158     -1.440362      -0.27029
[-14.10921661]
[-6.94602661]
negative
為
YES
   word  all   pos   neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
24    為  107  49.0  58.0  0.457944  0.542056     -0.781009     -0.612386
[-14.89022514]
[-7.55841243]
negative
本来
YES
    word  all  pos  neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
861   本来    2  2.0  0.0       1.0      0.01           0.0      -4.60517
[-14.89022514]
[-12.16358262]
negative
の
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
1    の  891  449.0  442.0  0.503928  0.496072     -0.685322     -0.701035
[-15.57554668]
[-12.86461717]
negative
長所
だ
YES
  word  all    pos    neg  pos_prob  neg_prob  log_pos_prob  log_neg_prob
4    だ  682  359.0  323.0  0.526393 

In [303]:
result

,gender,year,kind,opinion,predict_result
0,female,2nd,positive,多種多様な人に出会えること。大学という存在そのもの自体が、様々な人が集まる場所であるが、SF...,positive
1,male,3rd,negative,社会常識の欠如した学生がしばしば散見される。AO入試制度等を活用し、多彩なバックグラウンドを...,negative
2,male,2nd,positive,たくさんの研究会が存在し、１年生２年生という低学年のうちから所属することができ、その分野のラ...,positive
3,male,2nd,negative,SFCはよく慶應大学と隔離されて考えることが多く、慶應義塾大学というくくりの中でかなり存在感...,negative
4,male,1st,positive,とにかく人材の宝の山だと思う。多種多様なバックグラウンドを持った個性的で面白い人達がキャンパ...,positive
5,male,2nd,negative,SFC周辺には牛や豚の農場があり、その家畜の糞便から発生する匂いが風に乗って大学内まで運ばれ...,negative
6,male,1st,positive,プログラミングが進級のために必修であり、将来に役立つような授業体系となっている。どんな人であ...,positive
7,female,2nd,negative,学べる分野は多様で自分で好きなように学ぶ事ができるが、反対にやりたい事がわからず、目的意識が...,positive
8,male,2nd,positive,立地がユニーク。キャンパス内にはたくさんの木がある。周りを見渡すと山がある。ネットの世界に入...,positive
9,male,1st,negative,キャンパス内を歩いていると異臭が漂うことが多くあります。かなりきついものです。豊かな自然が広...,negative


In [306]:
result['YES'] = [1 if result.loc[i]['kind'] == result.loc[i]['predict_result'] else 0 for i in range(len(result.index))]

In [307]:
result

,gender,year,kind,opinion,predict_result,YES
0,female,2nd,positive,多種多様な人に出会えること。大学という存在そのもの自体が、様々な人が集まる場所であるが、SF...,positive,1
1,male,3rd,negative,社会常識の欠如した学生がしばしば散見される。AO入試制度等を活用し、多彩なバックグラウンドを...,negative,1
2,male,2nd,positive,たくさんの研究会が存在し、１年生２年生という低学年のうちから所属することができ、その分野のラ...,positive,1
3,male,2nd,negative,SFCはよく慶應大学と隔離されて考えることが多く、慶應義塾大学というくくりの中でかなり存在感...,negative,1
4,male,1st,positive,とにかく人材の宝の山だと思う。多種多様なバックグラウンドを持った個性的で面白い人達がキャンパ...,positive,1
5,male,2nd,negative,SFC周辺には牛や豚の農場があり、その家畜の糞便から発生する匂いが風に乗って大学内まで運ばれ...,negative,1
6,male,1st,positive,プログラミングが進級のために必修であり、将来に役立つような授業体系となっている。どんな人であ...,positive,1
7,female,2nd,negative,学べる分野は多様で自分で好きなように学ぶ事ができるが、反対にやりたい事がわからず、目的意識が...,positive,0
8,male,2nd,positive,立地がユニーク。キャンパス内にはたくさんの木がある。周りを見渡すと山がある。ネットの世界に入...,positive,1
9,male,1st,negative,キャンパス内を歩いていると異臭が漂うことが多くあります。かなりきついものです。豊かな自然が広...,negative,1


In [312]:
check = list(result['YES'].values)
check.count(1)

51

In [313]:
print("accuracy is {}".format(check.count(1) / len(result.index)))

accuracy is 0.85


In [24]:
test_df.head(5)

,gender,year,kind,opinion
0,female,2nd,positive,多種多様な人に出会えること。大学という存在そのもの自体が、様々な人が集まる場所であるが、SF...
1,male,3rd,negative,社会常識の欠如した学生がしばしば散見される。AO入試制度等を活用し、多彩なバックグラウンドを...
2,male,2nd,positive,たくさんの研究会が存在し、１年生２年生という低学年のうちから所属することができ、その分野のラ...
3,male,2nd,negative,SFCはよく慶應大学と隔離されて考えることが多く、慶應義塾大学というくくりの中でかなり存在感...
4,male,1st,positive,とにかく人材の宝の山だと思う。多種多様なバックグラウンドを持った個性的で面白い人達がキャンパ...
